In [8]:
!pip install moviepy


Defaulting to user installation because normal site-packages is not writeable


In [75]:
from moviepy import VideoFileClip

video = VideoFileClip("C:/Users/HILCPS/Downloads/p8209.mp4")
video.audio.write_audiofile("C:/Users/HILCPS/Downloads/audiop8209.wav")


MoviePy - Writing audio in C:/Users/HILCPS/Downloads/audiop8209.wav


MoviePy - Done.


In [26]:
!pip install git+https://github.com/openai/whisper.git


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/openai/whisper.git to c:\users\hilcps\appdata\local\temp\pip-req-build-qs70vtfp
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git 'C:\Users\HILCPS\AppData\Local\Temp\pip-req-build-qs70vtfp'


In [25]:
# tsv to csv conversion

In [24]:
import csv

tsv_path = r"C:\Users\HILCPS\Downloads\audio.tsv"
csv_path = r"C:\Users\HILCPS\Downloads\transcript.csv"

with open(tsv_path, 'r', encoding='utf-8') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    rows = list(reader)

with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(rows)

print("Saved as CSV:", csv_path)


Saved as CSV: C:\Users\HILCPS\Downloads\transcript.csv


In [ ]:
# to remove unnecessary details and to save only required time stamps and text into json file 

In [76]:
import json

# Load the full whisper JSON output
with open(r"C:\Users\HILCPS\Downloads\audiop8209.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Extract only timestamps and text
minimal = [
    {
        "start": round(seg["start"], 2),
        "end": round(seg["end"], 2),
        "text": seg["text"].strip()
    }
    for seg in data["segments"]
]

# Save the minimal JSON
with open(r"C:\Users\HILCPS\Downloads\transcript_clean_p8209.json", "w", encoding="utf-8") as f:
    json.dump(minimal, f, indent=4, ensure_ascii=False)

print("Cleaned transcript saved as transcript_clean_p8209.json")


Cleaned transcript saved as transcript_clean_p8209.json


In [44]:
#  What It Does:
# Loads your labeled JSON (with start, end, label)

# Prompts you on the terminal to select a label (e.g., WICKET)

# Filters all events with that label

# Extracts video segments from the full video based on timestamps

# Merges all clips for that label into one single output video

In [47]:
!pip install --upgrade moviepy


Defaulting to user installation because normal site-packages is not writeable


In [58]:
!pip install ffmpeg-python


Defaulting to user installation because normal site-packages is not writeable


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [81]:
import json
import os
import subprocess

# === Input Paths ===
# input_video = r"C:\Users\HILCPS\Downloads\p8208.mp4"
# json_path = r"C:\Users\HILCPS\Downloads\transcript_llama3_labels.json"
# output_dir = r"C:\Users\HILCPS\Downloads\clips_temp"
# output_merged = r"C:\Users\HILCPS\Downloads\extracted_SIX_ffmpeg.mp4"
input_video = r"C:\Users\HILCPS\Downloads\p8209.mp4"
json_path = r"C:\Users\HILCPS\Downloads\transcript_prompted_llama3_labels_p8209.json"
output_dir = r"C:\Users\HILCPS\Downloads\clips_temp"
output_merged = r"C:\Users\HILCPS\Downloads\extracted_prompted_FOUR_ffmpeg_p8209.mp4"
# === Create temp dir ===
os.makedirs(output_dir, exist_ok=True)

# === Load JSON ===
with open(json_path, "r", encoding="utf-8") as f:
    transcript = json.load(f)

# === Ask for label ===
label = input("🔍 Enter label to extract (e.g., SIX): ").strip().upper()

# === Filter entries ===
segments = [t for t in transcript if t["label"] == label]
print(f"✂️ Found {len(segments)} segments for label: {label}")

# === Trim clips ===
trimmed_files = []
for i, seg in enumerate(segments):
    start = float(seg["start"])
    duration = float(seg["end"]) - start
    clip_path = os.path.join(output_dir, f"clip_{i:03d}.mp4")
    trimmed_files.append(clip_path)
    ffmpeg_path = r"C:\ffmpeg-7.1.1-essentials_build\ffmpeg-7.1.1-essentials_build\bin\ffmpeg.exe" # Change this if your location is different
    cmd = [
        ffmpeg_path, "-y",
        "-i", input_video,
        "-ss", str(start),
        "-t", str(duration),
        "-c", "copy",
        clip_path
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"✅ Trimmed: {clip_path}")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed: {clip_path}")

# === Create file list for merge ===
list_path = os.path.join(output_dir, "merge_list.txt")
with open(list_path, "w", encoding="utf-8") as f:
    for clip in trimmed_files:
        f.write(f"file '{clip}'\n")

# === Merge clips ===
merge_cmd = [
    ffmpeg_path, "-y",
    "-f", "concat",
    "-safe", "0",
    "-i", list_path,
    "-c", "copy",
    output_merged
]

try:
    subprocess.run(merge_cmd, check=True)
    print(f"\n🎬 Merged video saved to: {output_merged}")
except subprocess.CalledProcessError:
    print("❌ Failed to merge clips.")


🔍 Enter label to extract (e.g., SIX):  FOUR


✂️ Found 1 segments for label: FOUR
✅ Trimmed: C:\Users\HILCPS\Downloads\clips_temp\clip_000.mp4

🎬 Merged video saved to: C:\Users\HILCPS\Downloads\extracted_prompted_FOUR_ffmpeg_p8209.mp4
